In [3]:
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import inspect
import pandas as pd
from sqlalchemy.sql import functions

# Challenge 1


In [4]:
connection_string = 'sqlite:///datasets/publications.db'
connection_string


'sqlite:///datasets/publications.db'

In [5]:
engine=create_engine(connection_string)
engine

Engine(sqlite:///datasets/publications.db)

In [6]:
inspector = inspect(engine)
inspector.get_table_names()

['authors',
 'discounts',
 'employee',
 'jobs',
 'pub_info',
 'publishers',
 'roysched',
 'sales',
 'stores',
 'titleauthor',
 'titles']

# Challenge 1

In [7]:
Authors=pd.read_sql_query('SELECT * FROM authors', engine)
Authors.head(1)


,au_id,au_lname,au_fname,phone,address,city,state,zip,contract
0,172-32-1176,White,Johnson,408 496-7223,10932 Bigge Rd.,Menlo Park,CA,94025,1


In [8]:
Discounts = pd.read_sql_query('SELECT * FROM discounts',engine)
Discounts.head(1)


,discounttype,stor_id,lowqty,highqty,discount
0,Initial Customer,None,NaN,NaN,10.5


In [9]:
Employee = pd.read_sql_query('SELECT * FROM employee', engine)
Employee.head(1)

,emp_id,fname,minit,lname,job_id,job_lvl,pub_id,hire_date
0,A-C71970F,Aria,,Cruz,10,87,1389,1991-10-26 00:00:00


In [10]:
jobs = pd.read_sql_query('SELECT * FROM jobs', engine)
jobs.head(1)

,job_id,job_desc,min_lvl,max_lvl
0,1,New Hire - Job not specified,10,10


In [11]:
pub_info = pd.read_sql_query('SELECT * FROM pub_info', engine)
pub_info.head(1)

,pub_id,logo,pr_info
0,0736,"b""GIF89a\xd3\x00\x1f\x00\xb3\x0f\x00\x00\x00\x...","This is sample text data for New Moon Books, p..."


In [12]:
publishers= pd.read_sql_query('SELECT * FROM publishers', engine)
publishers.head(1)

,pub_id,pub_name,city,state,country
0,0736,New Moon Books,Boston,MA,USA


In [13]:
roysched= pd.read_sql_query('SELECT * FROM roysched', engine)
roysched.head(1)

,title_id,lorange,hirange,royalty
0,BU1032,0,5000,10


In [14]:
sales=pd.read_sql_query('SELECT * FROM sales', engine)
sales.head(1)


,stor_id,ord_num,ord_date,qty,payterms,title_id
0,6380,6871,1994-09-14 00:00:00,5,Net 60,BU1032


In [15]:
stores=pd.read_sql_query('SELECT * FROM stores', engine)
stores.head(1)

,stor_id,stor_name,stor_address,city,state,zip
0,6380,Eric the Read Books,788 Catamaugus Ave.,Seattle,WA,98056


In [16]:
titleauthor=pd.read_sql_query('SELECT * FROM titleauthor', engine)
titleauthor.head(1)

,au_id,title_id,au_ord,royaltyper
0,172-32-1176,PS3333,1,100


In [17]:
titles=pd.read_sql_query('SELECT * FROM titles', engine)
titles.head(1)

,title_id,title,type,pub_id,price,advance,royalty,ytd_sales,notes,pubdate
0,BU1032,The Busy Executive's Database Guide,business,1389,19.99,5000.0,10.0,4095.0,An overview of available database systems with...,1991-06-12 00:00:00


In [18]:
# Step 1

query_1 ='''
SELECT authors.au_id AS 'Authors',
titles.title_id AS 'Titles',
titles.advance * titleauthor.royaltyper / 100 AS 'Advance',
titles.price * sales.qty * titles.royalty / 100 * titleauthor.royaltyper / 100 AS 'Royalty'

FROM authors
INNER JOIN titleauthor ON authors.au_id = titleauthor.au_id
INNER JOIN titles ON titleauthor.title_id = titles.title_id 
INNER JOIN sales ON titleauthor.title_id = sales.title_id
'''

In [19]:
df_ch1 = pd.read_sql_query(query_1, engine)
df_ch1.head(5)

,Authors,Titles,Advance,Royalty
0,172-32-1176,PS3333,2000,29.985
1,213-46-8915,BU1032,2000,3.998
2,213-46-8915,BU1032,2000,7.996
3,213-46-8915,BU2075,10125,25.116
4,238-95-7766,PC1035,7000,110.160


In [20]:
# Step 2



In [29]:
query_2 = '''
SELECT authors.au_id AS 'Authors',
titles.title_id AS 'Titles',
titles.advance * titleauthor.royaltyper / 100 AS 'Advance',
SUM (titles.price * sales.qty * titles.royalty / 100 * titleauthor.royaltyper / 100) AS 'Royalty'

FROM authors
INNER JOIN titleauthor ON authors.au_id = titleauthor.au_id
INNER JOIN titles ON titleauthor.title_id = titles.title_id 
LEFT JOIN sales ON titleauthor.title_id = sales.title_id

GROUP BY authors.au_id, titles.title_id 
'''

In [30]:
df_ch2 = pd.read_sql_query(query_2, engine)
df_ch2.head(5)

,Authors,Titles,Advance,Royalty
0,172-32-1176,PS3333,2000.0,29.985
1,213-46-8915,BU1032,2000.0,11.994
2,213-46-8915,BU2075,10125.0,25.116
3,238-95-7766,PC1035,7000.0,110.160
4,267-41-2394,BU1111,2000.0,11.950


In [31]:
#Step 3
query_3 = '''
SELECT authors.au_id AS 'Authors',
titles.title_id AS 'Titles',
SUM (titles.advance * titleauthor.royaltyper) / 100 AS 'Advance',
SUM (titles.price * sales.qty * titles.royalty / 100 * titleauthor.royaltyper) / 100 AS 'Royalty'


FROM authors
INNER JOIN titleauthor ON authors.au_id = titleauthor.au_id
INNER JOIN titles ON titleauthor.title_id = titles.title_id 
LEFT JOIN sales ON titleauthor.title_id = sales.title_id

GROUP BY authors.au_id, titles.title_id 
'''

In [32]:
df_ch3 = pd.read_sql_query(query_3, engine)
df_ch3.head()

,Authors,Titles,Advance,Royalty
0,172-32-1176,PS3333,2000.0,29.985
1,213-46-8915,BU1032,4000.0,11.994
2,213-46-8915,BU2075,10125.0,25.116
3,238-95-7766,PC1035,7000.0,110.160
4,267-41-2394,BU1111,2000.0,11.950


# Challenge 2